In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
import networkx as nx
from sklearn.preprocessing import StandardScaler


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Seting the directory 
%cd /content/drive/MyDrive/TP_M1_SID/Second_semestre/Compet_kaggle  

/content/drive/MyDrive/TP_M1_SID/Second_semestre/Compet_kaggle


In [ ]:
df = pd.read_csv("comments_students.csv")[['id','ups', 'link_id','name','parent_id']]
df.head() 

,id,ups,link_id,name,parent_id
0,cqug90j,3.0,t3_34f9rh,t1_cqug90j,t1_cqug2sr
1,cqug90k,3.0,t3_34fvry,t1_cqug90k,t3_34fvry
2,cqug90z,5.0,t3_34ffo5,t1_cqug90z,t1_cqu80zb
3,cqug91c,1.0,t3_34aqsn,t1_cqug91c,t1_cqtdj4m
4,cqug91e,101.0,t3_34f9rh,t1_cqug91e,t1_cquc4rc


In [ ]:
df.shape

(4234970, 5)

In [ ]:
df.sort_values("ups", ascending=False) 

,id,ups,link_id,name,parent_id
1336877,cr56nez,6761.0,t3_35jfjt,t1_cr56nez,t1_cr56jvy
1377068,cr5gn52,6736.0,t3_35l896,t1_cr5gn52,t3_35l896
1156164,cr3imvn,5992.0,t3_35dppz,t1_cr3imvn,t3_35dppz
1792557,cr8p5z9,5927.0,t3_35wytd,t1_cr8p5z9,t3_35wytd
417240,cqxx3fj,5849.0,t3_34t8yf,t1_cqxx3fj,t1_cqxvi1h
...,...,...,...,...,...
4234965,crrbelu,NaN,t3_37y5rx,t1_crrbelu,t3_37y5rx
4234966,crrbelv,NaN,t3_37y5rx,t1_crrbelv,t3_37y5rx
4234967,crrbemp,NaN,t3_380jx2,t1_crrbemp,t3_380jx2
4234968,crrbenh,NaN,t3_37yawp,t1_crrbenh,t3_37yawp


#Creating Graph

In [ ]:
G = nx.DiGraph()

In [ ]:
G.add_nodes_from(df.link_id, type="link")

In [ ]:
%%time
G.add_nodes_from(df.name, type="comment") 

CPU times: user 8.53 s, sys: 2.37 s, total: 10.9 s
Wall time: 10.9 s


In [ ]:
%time
G.add_edges_from(df[["name","parent_id"]].values, link_type="parent") 

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


In [ ]:
print(G.number_of_nodes())
print(G.number_of_edges())

4396503
4234970


# Successors

##### It calculates the number of nodes at which point a comment, here as a comment either does not respond to any comment or to one and only one comment, then we have 0 or 1 as possible value. 

In [ ]:
df["successors_name"] = [len(list(G.successors(i))) for i in df["name"]]

# Predecessors

In [ ]:
df["predecessors_name"] = [len(list(G.predecessors(i))) for i in df["name"]]

In [ ]:
df.sort_values('ups', ascending= False).head(10)

,id,ups,link_id,name,parent_id,successors_name,predecessors_name,Ctr_degree_name,Ctr_cless_name,Ctr_Eigen_name,page_rk_name
1336877,cr56nez,6761.0,t3_35jfjt,t1_cr56nez,t1_cr56jvy,1,214,0.000049,0.000044,0.000332,0.000041
1377068,cr5gn52,6736.0,t3_35l896,t1_cr5gn52,t3_35l896,1,214,0.000049,0.000095,0.014722,0.000041
1156164,cr3imvn,5992.0,t3_35dppz,t1_cr3imvn,t3_35dppz,1,143,0.000033,0.000059,0.007909,0.000028
1792557,cr8p5z9,5927.0,t3_35wytd,t1_cr8p5z9,t3_35wytd,1,66,0.000015,0.000034,0.003867,0.000013
417240,cqxx3fj,5849.0,t3_34t8yf,t1_cqxx3fj,t1_cqxvi1h,1,10,0.000003,0.000027,0.004945,0.000002
1785001,cr8nddw,5776.0,t3_35wytd,t1_cr8nddw,t3_35wytd,1,45,0.000010,0.000019,0.001196,0.000009
1377025,cr5gmqg,5767.0,t3_35l896,t1_cr5gmqg,t3_35l896,1,36,0.000008,0.000012,0.003540,0.000007
3073793,crih4mb,5741.0,t3_36zcka,t1_crih4mb,t3_36zcka,1,125,0.000029,0.000038,0.003115,0.000024
66466,cquzbrm,5720.0,t3_34hywb,t1_cquzbrm,t3_34hywb,1,190,0.000043,0.000038,0.000640,0.000037
1656847,cr7p814,5710.0,t3_35tpuk,t1_cr7p814,t3_35tpuk,1,120,0.000028,0.000038,0.002173,0.000023


# Centrality

In [ ]:
def val_ctr(id):
  return centrality[id] 

### 1. Degree of centrality

##### I use the principle that the more comments that respond to a comment, the higher the score in absolute value.

In [ ]:
centrality = nx.degree_centrality(G)

In [ ]:
df['Ctr_degree_name'] = df['name'].apply(func = val_ctr) 
del centrality

### 2. Closeness
I calculate the importance of a comment in relation to its proximity to the comments which have a high score.

In [ ]:
centrality = nx.algorithms.closeness_centrality(G) 

In [ ]:
df['Ctr_cless_name'] = df['name'].apply(func = val_ctr)
del centrality 

###3. Eigen vector
###### I suppose that the more a comment responds to comments that have good scores, the higher its value. 



In [ ]:
%%time
centrality = nx.eigenvector_centrality(G) 

CPU times: user 5min 47s, sys: 2.67 s, total: 5min 49s
Wall time: 5min 51s


In [ ]:
df['Ctr_Eigen_name'] = df['name'].apply(func = val_ctr)
del centrality 

#Prestiges

As we have a directed graph, this part makes more sense to influence the score. Prestige calculates the score of a comment based on the number of comments that were made afterwards, even if they did not respond directly to the comment. For example comments that respond to a comment that has responded to another comment.

####1. Page Rank
I calculate the score of a comment based on the number of its predecessors and the importance that these predecessors have.

In [ ]:
def val_ctr(id):
  return pgR[id]

In [ ]:
pgR = nx.pagerank_scipy(G)

In [ ]:
df['page_rk_name'] = df['name'].apply(func = val_ctr)
del pgR 

##2. Hits

I calculate the score of a comment based on the number of "good" comment that it responds, or number of "good" comments which respond to it. 

In [ ]:
%%time
hits=nx.hits_scipy(G)

CPU times: user 42.2 s, sys: 1.17 s, total: 43.4 s
Wall time: 43.3 s


In [ ]:
def calcul_hits(name):
    return hits[h][name]

In [ ]:

h=0
df['hubs_name']=df['name'].apply(func=calcul_hits)

h=1
df['authority_name']=df['name'].apply(func=calcul_hits)
df.head() 

,id,ups,link_id,name,parent_id,successors_name,predecessors_name,Ctr_degree_name,Ctr_cless_name,Ctr_Eigen_name,page_rk_name,hubs_name,authority_name
0,cqug90j,3.0,t3_34f9rh,t1_cqug90j,t1_cqug2sr,1,0,2.274536e-07,0.000000e+00,1.838186e-09,4.122141e-08,5.030285e-77,0.000000e+00
1,cqug90k,3.0,t3_34fvry,t1_cqug90k,t3_34fvry,1,0,2.274536e-07,0.000000e+00,1.838186e-09,4.122141e-08,1.777681e-26,0.000000e+00
2,cqug90z,5.0,t3_34ffo5,t1_cqug90z,t1_cqu80zb,1,0,2.274536e-07,0.000000e+00,1.838186e-09,4.122141e-08,7.675606e-66,0.000000e+00
3,cqug91c,1.0,t3_34aqsn,t1_cqug91c,t1_cqtdj4m,1,0,2.274536e-07,0.000000e+00,1.838186e-09,4.122141e-08,2.160491e-67,0.000000e+00
4,cqug91e,101.0,t3_34f9rh,t1_cqug91e,t1_cquc4rc,1,1,4.549071e-07,3.032714e-07,2.830806e-07,2.345569e-07,3.296648e-72,1.547869e-72


# Connectivity
It measures how connected nodes with certain degrees are

In [ ]:
n = list(G.nodes)


In [ ]:
## Calculating the nodes dregree of connectivity 
def dg_con(n):
  return list(nx.networkx.algorithms.assortativity.average_degree_connectivity(G, nodes = [n]).values())[0]

In [ ]:
df['dg_connectivity_name'] = df['name'].apply(dg_con)

In [ ]:
df.sort_values('ups', ascending=False).head(20) 

# Calculating some values for the parent_id

## 1.Successors and  predecessors

In [ ]:
%%time 
df["successors_prt_id"] = [len(list(G.successors(i))) for i in df["parent_id"]]

CPU times: user 5.44 s, sys: 48.8 ms, total: 5.49 s
Wall time: 5.51 s


In [ ]:
df["predecessors_prt_id"] = [len(list(G.predecessors(i))) for i in df["parent_id"]]

### PageRank

In [ ]:
def val_ctr(id):
  return pgR[id]

In [ ]:
pgR = nx.pagerank_scipy(G)

In [ ]:
df['page_rk_prt_id'] = df['parent_id'].apply(func = val_ctr)
del pgR 

### HITS

In [ ]:
%%time
hits=nx.hits_scipy(G)

CPU times: user 38.7 s, sys: 962 ms, total: 39.7 s
Wall time: 39.6 s


In [ ]:
def calcul_hits(name):
    return hits[h][name]

In [ ]:
%%time
h=0
df['hubs_prt_id']=df['parent_id'].apply(func=calcul_hits)

h=1
df['authority_prt_id']=df['parent_id'].apply(func=calcul_hits)
df.head() 

CPU times: user 4.34 s, sys: 13.5 ms, total: 4.35 s
Wall time: 4.35 s


## Scaling data 
#### I are going to center and reduce some variables

In [ ]:
scaler = StandardScaler()
df.iloc[:,5:]=scaler.fit_transform(df.iloc[:,5:])
df 

,id,ups,link_id,name,parent_id,successors_name,predecessors_name,Ctr_degree_name,Ctr_cless_name,Ctr_Eigen_name,page_rk_name,hubs_name,authority_name,dg_connectivity_name,successors_prt_id,predecessors_prt_id,page_rk_prt_id,hubs_prt_id,authority_prt_id
0,cqug90j,3.0,t3_34f9rh,t1_cqug90j,t1_cqug2sr,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-0.085552,-0.000493,-0.186449,-1.206389,-0.193494,-0.193494,-0.01751,-0.085552
1,cqug90k,3.0,t3_34fvry,t1_cqug90k,t3_34fvry,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-0.085552,-0.000493,0.324905,-1.206389,0.313289,0.313289,-0.01751,-0.085552
2,cqug90z,5.0,t3_34ffo5,t1_cqug90z,t1_cqu80zb,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-0.085552,-0.000493,-0.185031,-1.206389,-0.192089,-0.192089,-0.01751,-0.085552
3,cqug91c,1.0,t3_34aqsn,t1_cqug91c,t1_cqtdj4m,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-0.085552,-0.000493,-0.185385,-1.206389,-0.192440,-0.192440,-0.01751,-0.085552
4,cqug91e,101.0,t3_34f9rh,t1_cqug91e,t1_cquc4rc,0.0,0.132467,0.132467,0.146519,-0.061107,0.132467,-0.085552,-0.000493,-0.186449,-1.206389,-0.193143,-0.193143,-0.01751,-0.085552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234965,crrbelu,NaN,t3_37y5rx,t1_crrbelu,t3_37y5rx,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-0.085552,-0.000493,0.343699,-1.206389,0.331915,0.331915,-0.01751,-0.085552
4234966,crrbelv,NaN,t3_37y5rx,t1_crrbelv,t3_37y5rx,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-0.085552,-0.000493,0.343699,-1.206389,0.331915,0.331915,-0.01751,-0.085552
4234967,crrbemp,NaN,t3_380jx2,t1_crrbemp,t3_380jx2,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-0.085552,-0.000493,-0.183612,-1.206389,-0.190683,-0.190683,-0.01751,-0.085552
4234968,crrbenh,NaN,t3_37yawp,t1_crrbenh,t3_37yawp,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-0.085552,-0.000493,0.444055,-1.206389,0.431374,0.431374,-0.01751,-0.085552


### Saving a copy 

In [ ]:
df.to_csv('df_links_scaled.csv',  index= False)